In [2]:
import pandas as pd
import numpy as np

p = pd.read_csv('/content/products_data.csv') #products
s = pd.read_csv('/content/sales_data.csv') #sales

print('Вид датафрейма p:')
print(p.head())
print('Вид датафрейма s:')
print(s.head())



Вид датафрейма p:
   product_id                 product_name    category manufacturer
0         101       Ноутбук HP Pavilion 15    Ноутбуки            A
1         102  Смартфон Samsung Galaxy S21   Смартфоны            B
2         103       Планшет Apple iPad Air    Планшеты            C
3         104     Наушники Sony WH-1000XM4    Наушники            D
4         105         Телевизор LG OLED CX  Телевизоры            E
Вид датафрейма s:
   order_id  product_id  customer_id  order_date  quantity  price_per_unit  \
0         1         101         1001  2022-02-15         2             500   
1         2         102         1002  2022-03-20         1             800   
2         3         103         1003  2022-03-25         3             300   
3         4         104         1001  2022-04-10         2             400   
4         5         105         1004  2022-04-15         1             700   

   total_price payment_method  region  
0         1000          Карта   Север  
1      

In [3]:
print('Пропуски в products:')
print(p.isna().any())
print('Пропуски в sales:')
print(s.isna().any())

Пропуски в products:
product_id      False
product_name    False
category        False
manufacturer    False
dtype: bool
Пропуски в sales:
order_id          False
product_id        False
customer_id       False
order_date        False
quantity          False
price_per_unit    False
total_price       False
payment_method    False
region            False
dtype: bool


1. В каком регионе было продано наибольшее количество продуктов категории "Смартфоны"?

In [4]:
#Наиболее простой способ - объединить датафреймы
df = s.merge(p, on='product_id')
df1 = df[(df['category'] == 'Смартфоны')].groupby('region')['quantity'].agg(sum).sort_values(ascending = False).head(1)
df1

region
Восток    4
Name: quantity, dtype: int64

In [5]:
#Также можно попробовать без использования df_merged
df2 = p[(p['category'] == 'Смартфоны')]
a = list(df2['product_id'].values)
s_smartphones = s[s.product_id.isin(a)].groupby('region')['quantity'].agg(sum).sort_values(ascending = False).head(1)
s_smartphones

region
Восток    4
Name: quantity, dtype: int64

2. Какова разница между максимальной и минимальной средней ценой продуктов категории "Ноутбуки" между всеми производителями? Ответ округлите до целых.

In [14]:
df3 = df[(df['category'] == 'Ноутбуки')].groupby('manufacturer')['price_per_unit'].mean()
prices = [i for i in df3.values]
round(max(prices) - min(prices))

200

3. Какой покупатель совершил заказы на наибольшую сумму по всем временам?

In [7]:
customer_max = s.groupby('customer_id')['total_price'].agg(sum).sort_values(ascending = False)
customer_max.idxmax()


1003

4. Какова доля продаж продуктов категории "Планшеты" от общей выручки за все время? Ответ округлите до 2 знака после запятой. Разделитель разряда - точка.

In [8]:
total_sales_pl = df[(df['category'] == 'Планшеты')]['total_price'].sum()
round(total_sales_pl/df['total_price'].sum(),2)


0.25

5. Какова разница в средней цене продуктов категории "Смартфоны" между двумя самыми крупными регионами по продажам? Ответ округлите до целых.

In [23]:
top_regions = df.groupby('region')['total_price'].sum().sort_values(ascending = False).head(2)
diff_smartphones = df[ (df['category'] == 'Смартфоны') & (df['region'].isin(top_regions.index)) ].groupby('region')['price_per_unit'].mean()
#так как мы знаем что diff_smartphones содержит лишь два значения в столбце цен, то можем смело выбрать значения по индексам
abs(diff_smartphones.iloc[0] - diff_smartphones.iloc[1])

275.0

6. Сколько в среднем уникальных категорий продуктов было продано по всем регионам?

In [42]:
df.groupby('region')['category'].nunique().mean()

5.0

7. Какой производитель представлен в наибольшем количестве различных категорий продуктов?

In [50]:
df.groupby('manufacturer')['category'].nunique().idxmax()

'E'